In [24]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [25]:
v = '10-1'

In [26]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [27]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [30]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/11/22 18:22:53] 603994771.py:9 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/11/22 18:23:01] 603994771.py:10 in <module>
                    calc_metric(xc, 'reg_pred'): 0.9135889034597657
[08/11/22 18:23:08] 603994771.py:11 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9141593913749759
[08/11/22 18:23:16] 603994771.py:12 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.902516912439078


0.902516912439078

In [31]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  # if prob.max() < 10:
  if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/11/22 18:23:25] 2337548337.py:27 in <module>
                    xpc['mark'].mean(): 0.5019261406823974
[08/11/22 18:23:34] 2337548337.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9026518550887598


0.9026518550887598

In [32]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=None):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  # x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  # x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  # topn = topn or len(idxes)
  topn = topn or 10
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        # x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  # diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = min(10, len(x_['cls_pred_ori']))
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  # x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  # x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  # x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        # x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  # diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  # diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  # diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x, topn=3) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(p_feats)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    # diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    # diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    # diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    # diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [33]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [34]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [35]:
gezi.set_fold(df, 5, 'ancestor_id')

In [36]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

114

In [37]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [38]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [39]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [40]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2522911	test: 0.2522911	test1: 0.2522307	best: 0.2522307 (0)	total: 55.2ms	remaining: 9m 11s
100:	learn: 0.0657628	test: 0.0657628	test1: 0.0668600	best: 0.0668600 (100)	total: 3.79s	remaining: 6m 12s
200:	learn: 0.0588901	test: 0.0588901	test1: 0.0602124	best: 0.0602124 (200)	total: 7.38s	remaining: 5m 59s
300:	learn: 0.0581114	test: 0.0581114	test1: 0.0594910	best: 0.0594910 (300)	total: 10.9s	remaining: 5m 50s
400:	learn: 0.0574170	test: 0.0574170	test1: 0.0588524	best: 0.0588524 (400)	total: 14.6s	remaining: 5m 48s
500:	learn: 0.0568868	test: 0.0568868	test1: 0.0583773	best: 0.0583773 (500)	total: 18.2s	remaining: 5m 44s
600:	learn: 0.0565586	test: 0.0565586	test1: 0.0581108	best: 0.0581108 (600)	total: 21.8s	remaining: 5m 41s
700:	learn: 0.0562972	test: 0.0562972	test1: 0.0579122	best: 0.0579122 (700)	total: 25.4s	remaining: 5m 37s
800:	learn: 0.0560962	test: 0.0560962	test1: 0.0577572	best: 0.0577572 (800)	total: 29.1s	remaining: 5m 34s
900:	learn: 0.0559113	test: 0.0

[08/11/22 18:30:26] 3198672829.py:8 in <module>
                    cbt_score: 0.9199312947996656


0.9199312947996656

In [41]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [42]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          # 'min_data_in_leaf': 300,
          'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [43]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=500)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.064233 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26660
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 114
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[100]	training's l1: 0.0667349	valid_1's l1: 0.0677891
[200]	training's l1: 0.0595607	valid_1's l1: 0.0607967
[300]	training's l1: 0.0586342	valid_1's l1: 0.0598966
[400]	training's l1: 0.0577119	valid_1's l1: 0.0590241
[500]	training's l1: 0.0571363	valid_1's l1: 0.0585103
[600]	training's l1: 0.0567232	valid_1's l1: 0.0581622
[700]	training's l1: 0.0564445	valid_1's l1: 0.057924
[800]	training's l1: 0.0563155	valid_1's l1: 0.0578205
[900]	training's l1: 0.0561609	valid_1's l1: 0.0577001
[1000]	training's l1: 0.0560563	valid_1's l1: 0.0576236
[1100]	training's l1: 0.0559411	valid_1'

[08/11/22 18:35:18] 176747773.py:12 in <module>
                    lgb_score: 0.920193588869891


0.920193588869891

In [44]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [45]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    20000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=200)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/11/22 18:35:26] 1820686574.py:21 in <module>
                    cbt_score: 0.9199312947996656
[08/11/22 18:36:04] 1820686574.py:36 in <module>
                    lgb_score: 0.920193588869891
[08/11/22 18:36:06] 1820686574.py:38 in <module>
                    score: 0.9203618490564346
[08/11/22 18:36:06] 1820686574.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9199312947996656
                    np.asarray(lgb_scores).mean(): 0.920193588869891
                    np.asarray(scores).mean(): 0.9203618490564346


0:	learn: 0.2523574	test: 0.2523574	test1: 0.2522460	best: 0.2522460 (0)	total: 46.1ms	remaining: 7m 40s
500:	learn: 0.0573135	test: 0.0573135	test1: 0.0565482	best: 0.0565482 (500)	total: 17.6s	remaining: 5m 33s
1000:	learn: 0.0562366	test: 0.0562366	test1: 0.0556926	best: 0.0556926 (1000)	total: 35.4s	remaining: 5m 18s
1500:	learn: 0.0557811	test: 0.0557811	test1: 0.0554061	best: 0.0554061 (1500)	total: 53.2s	remaining: 5m 1s
2000:	learn: 0.0554059	test: 0.0554059	test1: 0.0552363	best: 0.0552363 (2000)	total: 1m 10s	remaining: 4m 43s
2500:	learn: 0.0551061	test: 0.0551061	test1: 0.0551399	best: 0.0551387 (2485)	total: 1m 28s	remaining: 4m 24s
3000:	learn: 0.0548090	test: 0.0548090	test1: 0.0550573	best: 0.0550573 (3000)	total: 1m 45s	remaining: 4m 6s
3500:	learn: 0.0546033	test: 0.0546033	test1: 0.0550183	best: 0.0550180 (3497)	total: 2m 3s	remaining: 3m 49s
4000:	learn: 0.0543972	test: 0.0543972	test1: 0.0549796	best: 0.0549796 (4000)	total: 2m 23s	remaining: 3m 35s
4500:	learn: 0.

[08/11/22 18:42:54] 1820686574.py:21 in <module>
                    cbt_score: 0.9227714390899486


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060119 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26660
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 114
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0575831	valid_1's l1: 0.0568251
[1000]	training's l1: 0.0563875	valid_1's l1: 0.0557621
[1500]	training's l1: 0.0560149	valid_1's l1: 0.0554966
[2000]	training's l1: 0.0558062	valid_1's l1: 0.055361
[2500]	training's l1: 0.0556531	valid_1's l1: 0.0552784
[3000]	training's l1: 0.055526	valid_1's l1: 0.0552102
[3500]	training's l1: 0.0554087	valid_1's l1: 0.0551624
[4000]	training's l1: 0.0553011	valid_1's l1: 0.0551155
[4500]	training's l1: 0.0552071	valid_1's l1: 0.0550709
[5000]	training's l1: 0.055136	valid_1's l1: 0.0550519
[5500]	training's l1: 0.0550383	va

...

[08/11/22 19:05:27] 1820686574.py:36 in <module>
                    lgb_score: 0.9249279677698254
[08/11/22 19:05:29] 1820686574.py:38 in <module>
                    score: 0.9251668940708964
[08/11/22 19:05:29] 1820686574.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9224078693444893
                    np.asarray(lgb_scores).mean(): 0.922444504240303
                    np.asarray(scores).mean(): 0.9226853119385242


0:	learn: 0.2523023	test: 0.2523023	test1: 0.2523830	best: 0.2523830 (0)	total: 52.1ms	remaining: 8m 40s
500:	learn: 0.0567981	test: 0.0567981	test1: 0.0583059	best: 0.0583059 (500)	total: 19.5s	remaining: 6m 8s
1000:	learn: 0.0557991	test: 0.0557991	test1: 0.0574280	best: 0.0574280 (1000)	total: 39.5s	remaining: 5m 54s
1500:	learn: 0.0553406	test: 0.0553406	test1: 0.0571038	best: 0.0571038 (1500)	total: 59.2s	remaining: 5m 35s
2000:	learn: 0.0550149	test: 0.0550149	test1: 0.0569304	best: 0.0569304 (2000)	total: 1m 19s	remaining: 5m 16s
2500:	learn: 0.0547313	test: 0.0547313	test1: 0.0568262	best: 0.0568262 (2500)	total: 1m 38s	remaining: 4m 55s
3000:	learn: 0.0544678	test: 0.0544678	test1: 0.0567633	best: 0.0567633 (3000)	total: 1m 57s	remaining: 4m 34s
3500:	learn: 0.0542125	test: 0.0542125	test1: 0.0566971	best: 0.0566956 (3484)	total: 2m 17s	remaining: 4m 14s
4000:	learn: 0.0539989	test: 0.0539989	test1: 0.0566564	best: 0.0566563 (3989)	total: 2m 36s	remaining: 3m 55s
4500:	learn: 

[08/11/22 19:11:13] 1820686574.py:21 in <module>
                    cbt_score: 0.9155848372221671


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.063981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26658
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 114
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0571567	valid_1's l1: 0.0585787
[1000]	training's l1: 0.0560298	valid_1's l1: 0.0575959
[1500]	training's l1: 0.0556775	valid_1's l1: 0.057308
[2000]	training's l1: 0.0554924	valid_1's l1: 0.0571727
[2500]	training's l1: 0.0553603	valid_1's l1: 0.0570757
[3000]	training's l1: 0.0552347	valid_1's l1: 0.0569955
[3500]	training's l1: 0.0551339	valid_1's l1: 0.0569398
[4000]	training's l1: 0.0550228	valid_1's l1: 0.0568938
[4500]	training's l1: 0.0549438	valid_1's l1: 0.0568548
[5000]	training's l1: 0.0548434	valid_1's l1: 0.0568053
[5500]	training's l1: 0.0547686	

[08/11/22 19:15:05] 1820686574.py:36 in <module>
                    lgb_score: 0.9154757773825879
[08/11/22 19:15:07] 1820686574.py:38 in <module>
                    score: 0.9157116357103524
[08/11/22 19:15:07] 1820686574.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9210432629200248
                    np.asarray(lgb_scores).mean(): 0.92105075886876
                    np.asarray(scores).mean(): 0.9212905766928898


In [46]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614